In [1]:
from sklearn.model_selection import train_test_split
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context
import numpy as np
import pandas as pd

frame = pd.read_excel("data_gdm_discrete_BMI_merged.xlsx", engine="openpyxl")
frame.oDM = (frame.oDM == 2).astype(int)
for name in frame.columns:
    frame[name] = frame[name].astype('category')

print (frame)
print (frame.oDM.value_counts())

     AgeCat BMICat Race Education GravCat SmokeCat1 SmokeCat2 oDM
0         3      0    1         3       2         1         1   0
1         1      2    2         2       1         1         2   0
2         0      0    2         1       1         2         2   0
3         2      2    3         3       2         2         2   0
4         1      0    3         4       1         2         2   0
...     ...    ...  ...       ...     ...       ...       ...  ..
9215      2      1    1         4       2         1         1   0
9216      1      0    3         3       2         2         2   0
9217      0      0    3         1       2         2         2   0
9218      2      1    1         2       3         1         1   0
9219      3      2    1         5       1         1         1   0

[9220 rows x 8 columns]
0    8832
1     388
Name: oDM, dtype: int64


In [2]:
from sklearn.model_selection import train_test_split

train_data = frame.to_numpy()
names = frame.columns.tolist()

train, test = train_test_split(pd.get_dummies(frame), stratify = frame['oDM'].tolist(), random_state = 0)
print ("%d in train, %d in test" % (len(train), len(test)))

6915 in train, 2305 in test


In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules


frequent_itemsets = fpgrowth(train, use_colnames=True, min_support = 0.5)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold = 0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(oDM_0),(SmokeCat2_2),0.957918,0.824006,0.790022,0.824728,1.000877,0.000692,1.004122
1,(SmokeCat2_2),(oDM_0),0.824006,0.957918,0.790022,0.958757,1.000877,0.000692,1.020365
2,(GravCat_1),(oDM_0),0.744469,0.957918,0.714244,0.959402,1.001549,0.001105,1.036557
3,(oDM_0),(GravCat_1),0.957918,0.744469,0.714244,0.745622,1.001549,0.001105,1.004534
4,(GravCat_1),(SmokeCat2_2),0.744469,0.824006,0.622993,0.836830,1.015563,0.009547,1.078593
5,(SmokeCat2_2),(GravCat_1),0.824006,0.744469,0.622993,0.756055,1.015563,0.009547,1.047495
6,"(GravCat_1, oDM_0)",(SmokeCat2_2),0.714244,0.824006,0.598698,0.838226,1.017258,0.010157,1.087904
7,"(GravCat_1, SmokeCat2_2)",(oDM_0),0.622993,0.957918,0.598698,0.961003,1.003221,0.001922,1.079114
8,"(oDM_0, SmokeCat2_2)",(GravCat_1),0.790022,0.744469,0.598698,0.757825,1.017941,0.010552,1.055154
9,(GravCat_1),"(oDM_0, SmokeCat2_2)",0.744469,0.790022,0.598698,0.804196,1.017941,0.010552,1.072389


In [4]:
confidences = []
for i, row in rules.iterrows():
    acondition = None
    for name in row.antecedents:
        if acondition is None:
            acondition = (test[name] == 1)
        else:
            acondition = acondition & (test[name] == 1)
    
    ccondition = None
    for name in row.consequents:
        if ccondition is None:
            ccondition = (test[name] == 1)
        else:
            ccondition = ccondition & (test[name] == 1)
    
    confidences.append (len(test[acondition & ccondition]) / len(test[acondition]))

print ("test condidence = %.2f, alen = %.2f, clen = %.2f" % (np.mean(confidences), 
                                                            rules.antecedents.apply(len).mean(), 
                                                            rules.consequents.apply(len).mean()))

test condidence = 0.87, alen = 1.29, clen = 1.14
